# Run Any Kind of OLS Regression (ANOVA, GLM, etc.)

### Authors: Calvin Howard.

#### Last updated: February 1, 2025

Use this to run/test a statistical model (e.g., regression or T-tests) on a spreadsheet.

Notes:
- To best use this notebook, you should be familar with GLM design and Contrast Matrix design. See this webpage to get started:
[FSL's GLM page](https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/GLM)

Prepare output directory

In [2]:
# Specify where you want to save your results to
out_dir = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/raynor_network_mapping/results/corbetta_cluster/umapsV2/cluster_unique'

# 00 - Import CSV with All Data
**The CSV is expected to be in this format**
- ID and absolute paths to niftis are critical
```
+-----+----------------------------+--------------+--------------+--------------+
| ID  | Nifti_File_Path            | Covariate_1  | Covariate_2  | Covariate_3  |
+-----+----------------------------+--------------+--------------+--------------+
| 1   | /path/to/file1.nii.gz      | 0.5          | 1.2          | 3.4          |
| 2   | /path/to/file2.nii.gz      | 0.7          | 1.4          | 3.1          |
| 3   | /path/to/file3.nii.gz      | 0.6          | 1.5          | 3.5          |
| 4   | /path/to/file4.nii.gz      | 0.9          | 1.1          | 3.2          |
| ... | ...                        | ...          | ...          | ...          |
+-----+----------------------------+--------------+--------------+--------------+
```

Import Data

In [3]:
# Specify the path to your CSV file containing NIFTI paths
input_csv_path = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/raynor_network_mapping/results/corbetta_cluster/umapsV2/cluster_results.csv'
sheet = None

In [4]:
from calvin_utils.permutation_analysis_utils.statsmodels_palm import CalvinStatsmodelsPalm
# Instantiate the PalmPrepararation class
cal_palm = CalvinStatsmodelsPalm(input_csv_path=input_csv_path, output_dir=out_dir, sheet=sheet)
# Call the process_nifti_paths method
data_df = cal_palm.read_data()
data_df

,sample_id,umap_x,umap_y,umap_z,cluster_label,cluster_probability,cluster_persistence,Nifti_File_Path,variable
0,118,34.546182,59.652836,24.727279,0,0.399402,0.249057,/Users/cu135/Partners HealthCare Dropbox/Calvi...,motor_battery_complete_acute
1,1,34.301865,58.634522,23.944913,0,0.411194,0.249057,/Users/cu135/Partners HealthCare Dropbox/Calvi...,hvlt_delayt_acute
2,8,34.405446,58.520763,23.863964,0,0.406459,0.249057,/Users/cu135/Partners HealthCare Dropbox/Calvi...,nih3_3month
3,53,35.016279,57.507695,23.623492,0,0.469857,0.249057,/Users/cu135/Partners HealthCare Dropbox/Calvi...,nih5a_3month
4,165,35.647017,59.361605,24.305943,0,1.000000,0.249057,/Users/cu135/Partners HealthCare Dropbox/Calvi...,bvmt_hit_1year
...,...,...,...,...,...,...,...,...,...
188,147,37.457394,59.737321,23.210514,2,1.000000,0.067357,/Users/cu135/Partners HealthCare Dropbox/Calvi...,hvlt_perc_acute
189,74,34.971212,58.922549,23.737356,0,0.704706,0.249057,/Users/cu135/Partners HealthCare Dropbox/Calvi...,gdss_7_3month
190,188,35.048752,58.821596,23.787037,0,0.738952,0.249057,/Users/cu135/Partners HealthCare Dropbox/Calvi...,nih11_acute
191,68,34.861592,57.493570,23.644406,0,0.681240,0.249057,/Users/cu135/Partners HealthCare Dropbox/Calvi...,gdss_1_1year


# 01 - Preprocess Your Data

**Handle NANs**
- Set drop_nans=True is you would like to remove NaNs from data
- Provide a column name or a list of column names to remove NaNs from

In [5]:
data_df.columns

Index(['sample_id', 'umap_x', 'umap_y', 'umap_z', 'cluster_label',
       'cluster_probability', 'cluster_persistence', 'Nifti_File_Path',
       'variable'],
      dtype='object')

In [6]:
drop_list = ['cluster_label', 'Nifti_File_Path']

In [7]:
data_df = cal_palm.drop_nans_from_columns(columns_to_drop_from=drop_list)
display(data_df)

,sample_id,umap_x,umap_y,umap_z,cluster_label,cluster_probability,cluster_persistence,Nifti_File_Path,variable
0,118,34.546182,59.652836,24.727279,0,0.399402,0.249057,/Users/cu135/Partners HealthCare Dropbox/Calvi...,motor_battery_complete_acute
1,1,34.301865,58.634522,23.944913,0,0.411194,0.249057,/Users/cu135/Partners HealthCare Dropbox/Calvi...,hvlt_delayt_acute
2,8,34.405446,58.520763,23.863964,0,0.406459,0.249057,/Users/cu135/Partners HealthCare Dropbox/Calvi...,nih3_3month
3,53,35.016279,57.507695,23.623492,0,0.469857,0.249057,/Users/cu135/Partners HealthCare Dropbox/Calvi...,nih5a_3month
4,165,35.647017,59.361605,24.305943,0,1.000000,0.249057,/Users/cu135/Partners HealthCare Dropbox/Calvi...,bvmt_hit_1year
...,...,...,...,...,...,...,...,...,...
188,147,37.457394,59.737321,23.210514,2,1.000000,0.067357,/Users/cu135/Partners HealthCare Dropbox/Calvi...,hvlt_perc_acute
189,74,34.971212,58.922549,23.737356,0,0.704706,0.249057,/Users/cu135/Partners HealthCare Dropbox/Calvi...,gdss_7_3month
190,188,35.048752,58.821596,23.787037,0,0.738952,0.249057,/Users/cu135/Partners HealthCare Dropbox/Calvi...,nih11_acute
191,68,34.861592,57.493570,23.644406,0,0.681240,0.249057,/Users/cu135/Partners HealthCare Dropbox/Calvi...,gdss_1_1year


**Drop Row Based on Value of Column**

Define the column, condition, and value for dropping rows
- column = 'your_column_name'
- condition = 'above'  # Options: 'equal', 'above', 'below'

In [8]:
data_df.columns

Index(['sample_id', 'umap_x', 'umap_y', 'umap_z', 'cluster_label',
       'cluster_probability', 'cluster_persistence', 'Nifti_File_Path',
       'variable'],
      dtype='object')

Set the parameters for dropping rows

In [9]:
column = 'Dataset'  # The column you'd like to evaluate
condition = 'equal'  # The condition to check ('equal', 'above', 'below', 'not')
value = 'PD STN DBS' # The value to drop if found

In [10]:
# data_df, other_df = cal_palm.drop_rows_based_on_value(column, condition, value)
display(data_df)

,sample_id,umap_x,umap_y,umap_z,cluster_label,cluster_probability,cluster_persistence,Nifti_File_Path,variable
0,118,34.546182,59.652836,24.727279,0,0.399402,0.249057,/Users/cu135/Partners HealthCare Dropbox/Calvi...,motor_battery_complete_acute
1,1,34.301865,58.634522,23.944913,0,0.411194,0.249057,/Users/cu135/Partners HealthCare Dropbox/Calvi...,hvlt_delayt_acute
2,8,34.405446,58.520763,23.863964,0,0.406459,0.249057,/Users/cu135/Partners HealthCare Dropbox/Calvi...,nih3_3month
3,53,35.016279,57.507695,23.623492,0,0.469857,0.249057,/Users/cu135/Partners HealthCare Dropbox/Calvi...,nih5a_3month
4,165,35.647017,59.361605,24.305943,0,1.000000,0.249057,/Users/cu135/Partners HealthCare Dropbox/Calvi...,bvmt_hit_1year
...,...,...,...,...,...,...,...,...,...
188,147,37.457394,59.737321,23.210514,2,1.000000,0.067357,/Users/cu135/Partners HealthCare Dropbox/Calvi...,hvlt_perc_acute
189,74,34.971212,58.922549,23.737356,0,0.704706,0.249057,/Users/cu135/Partners HealthCare Dropbox/Calvi...,gdss_7_3month
190,188,35.048752,58.821596,23.787037,0,0.738952,0.249057,/Users/cu135/Partners HealthCare Dropbox/Calvi...,nih11_acute
191,68,34.861592,57.493570,23.644406,0,0.681240,0.249057,/Users/cu135/Partners HealthCare Dropbox/Calvi...,gdss_1_1year


**Standardize Data**
- Enter Columns you Don't want to standardize into a list
- group_col is the column containing a category for each dataset. It ensures standardization is performed within each group.

In [11]:
# Remove anything you don't want to standardize
cols_not_to_standardize = ['Nifti_File_Path', 'Subject']
group_col = 'Dataset'

In [12]:
# data_df = cal_palm.standardize_columns(cols_not_to_standardize, group_col=group_col)
data_df

,sample_id,umap_x,umap_y,umap_z,cluster_label,cluster_probability,cluster_persistence,Nifti_File_Path,variable
0,118,34.546182,59.652836,24.727279,0,0.399402,0.249057,/Users/cu135/Partners HealthCare Dropbox/Calvi...,motor_battery_complete_acute
1,1,34.301865,58.634522,23.944913,0,0.411194,0.249057,/Users/cu135/Partners HealthCare Dropbox/Calvi...,hvlt_delayt_acute
2,8,34.405446,58.520763,23.863964,0,0.406459,0.249057,/Users/cu135/Partners HealthCare Dropbox/Calvi...,nih3_3month
3,53,35.016279,57.507695,23.623492,0,0.469857,0.249057,/Users/cu135/Partners HealthCare Dropbox/Calvi...,nih5a_3month
4,165,35.647017,59.361605,24.305943,0,1.000000,0.249057,/Users/cu135/Partners HealthCare Dropbox/Calvi...,bvmt_hit_1year
...,...,...,...,...,...,...,...,...,...
188,147,37.457394,59.737321,23.210514,2,1.000000,0.067357,/Users/cu135/Partners HealthCare Dropbox/Calvi...,hvlt_perc_acute
189,74,34.971212,58.922549,23.737356,0,0.704706,0.249057,/Users/cu135/Partners HealthCare Dropbox/Calvi...,gdss_7_3month
190,188,35.048752,58.821596,23.787037,0,0.738952,0.249057,/Users/cu135/Partners HealthCare Dropbox/Calvi...,nih11_acute
191,68,34.861592,57.493570,23.644406,0,0.681240,0.249057,/Users/cu135/Partners HealthCare Dropbox/Calvi...,gdss_1_1year


# 02 - Define Your Formula

**Critical: the dependent (y) variable should always be the column with the neuroimaging files in it**

This is the formula relating outcome to predictors, and takes the form:
- y = B0 + B1 + B2 + B3 + . . . BN

It is defined using the columns of your dataframe instead of the variables above:
- 'Apples_Picked ~ hours_worked + owns_apple_picking_machine'

____
**ANOVA**
- Tests differences in means for one categorical variable.
- formula = 'Outcome ~ C(Group1)'

**2-Way ANOVA**
- Tests differences in means for two categorical variables without interaction.
- formula = 'Outcome ~ C(Group1) + C(Group2)'

**2-Way ANOVA with Interaction**
- Tests for interaction effects between two categorical variables.
- formula = 'Outcome ~ C(Group1) * C(Group2)'

**ANCOVA**
- Similar to ANOVA, but includes a covariate to control for its effect.
- formula = 'Outcome ~ C(Group1) + Covariate'

**2-Way ANCOVA**
- Extends ANCOVA with two categorical variables and their interaction, controlling for a covariate.
- formula = 'Outcome ~ C(Group1) * C(Group2) + Covariate'

**Multiple Regression**
- Assesses the impact of multiple predictors on an outcome.
- formula = 'Outcome ~ Predictor1 + Predictor2'

**Simple Linear Regression**
- Assesses the impact of a single predictor on an outcome.
- formula = 'Outcome ~ Predictor'

**MANOVA**
- Assesses multiple dependent variables across groups.
- Note: Not typically set up with a formula in statsmodels. Requires specialized functions.

____
Use the printout below to design your formula. 
- Left of the "~" symbol is the thing to be predicted. 
- Right of the "~" symbol are the predictors. 
- ":" indicates an interaction between two things. 
- "*" indicates and interactions AND it accounts for the simple effects too. 
- "+" indicates that you want to add another predictor. 

In [13]:
import pandas as pd
data_df = data_df.join(pd.get_dummies(data_df['cluster_label'].factorize()[0], prefix='cluster'))

In [14]:
data_df

,sample_id,umap_x,umap_y,umap_z,cluster_label,cluster_probability,cluster_persistence,Nifti_File_Path,variable,cluster_0,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6
0,118,34.546182,59.652836,24.727279,0,0.399402,0.249057,/Users/cu135/Partners HealthCare Dropbox/Calvi...,motor_battery_complete_acute,True,False,False,False,False,False,False
1,1,34.301865,58.634522,23.944913,0,0.411194,0.249057,/Users/cu135/Partners HealthCare Dropbox/Calvi...,hvlt_delayt_acute,True,False,False,False,False,False,False
2,8,34.405446,58.520763,23.863964,0,0.406459,0.249057,/Users/cu135/Partners HealthCare Dropbox/Calvi...,nih3_3month,True,False,False,False,False,False,False
3,53,35.016279,57.507695,23.623492,0,0.469857,0.249057,/Users/cu135/Partners HealthCare Dropbox/Calvi...,nih5a_3month,True,False,False,False,False,False,False
4,165,35.647017,59.361605,24.305943,0,1.000000,0.249057,/Users/cu135/Partners HealthCare Dropbox/Calvi...,bvmt_hit_1year,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,147,37.457394,59.737321,23.210514,2,1.000000,0.067357,/Users/cu135/Partners HealthCare Dropbox/Calvi...,hvlt_perc_acute,False,False,False,True,False,False,False
189,74,34.971212,58.922549,23.737356,0,0.704706,0.249057,/Users/cu135/Partners HealthCare Dropbox/Calvi...,gdss_7_3month,True,False,False,False,False,False,False
190,188,35.048752,58.821596,23.787037,0,0.738952,0.249057,/Users/cu135/Partners HealthCare Dropbox/Calvi...,nih11_acute,True,False,False,False,False,False,False
191,68,34.861592,57.493570,23.644406,0,0.681240,0.249057,/Users/cu135/Partners HealthCare Dropbox/Calvi...,gdss_1_1year,True,False,False,False,False,False,False


In [15]:
data_df.columns

Index(['sample_id', 'umap_x', 'umap_y', 'umap_z', 'cluster_label',
       'cluster_probability', 'cluster_persistence', 'Nifti_File_Path',
       'variable', 'cluster_0', 'cluster_1', 'cluster_2', 'cluster_3',
       'cluster_4', 'cluster_5', 'cluster_6'],
      dtype='object')

** the left side of the equation is expected to be called 'Nifti_File_Path'. This should be in your CSV as a column, spelled the same way. **

In [16]:
formula = "Nifti_File_Path ~ cluster_0 + cluster_1 + cluster_2 + cluster_3 + cluster_4 + cluster_5 + cluster_6"

# 02 - Visualize Your Design Matrix

This is the explanatory variable half of your regression formula
_______________________________________________________
Create Design Matrix: Use the create_design_matrix method. You can provide a list of formula variables which correspond to column names in your dataframe.

- design_matrix = palm.create_design_matrix(formula_vars=["var1", "var2", "var1*var2"])
- To include interaction terms, use * between variables, like "var1*var2".
- By default, an intercept will be added unless you set intercept=False
- **don't explicitly add the 'intercept' column. I'll do it for you.**
- If you want to compare specific datasets within a column, leave 'coerce_str'=False

In [17]:
# Define the design matrix
outcome_matrix, design_matrix = cal_palm.define_design_matrix(formula, data_df=data_df, voxelwise_variable_list=['Nifti_File_Path'], coerce_str=False)
design_matrix

,Intercept,cluster_0[T.True],cluster_1[T.True],cluster_2[T.True],cluster_3[T.True],cluster_4[T.True],cluster_5[T.True],cluster_6[T.True]
0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
188,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
189,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
190,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
191,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


# 03 - Visualize Your Dependent Variable

I have generated this for you based on the formula you provided

In [18]:
outcome_matrix

,Nifti_File_Path
0,/Users/cu135/Partners HealthCare Dropbox/Calvi...
1,/Users/cu135/Partners HealthCare Dropbox/Calvi...
2,/Users/cu135/Partners HealthCare Dropbox/Calvi...
3,/Users/cu135/Partners HealthCare Dropbox/Calvi...
4,/Users/cu135/Partners HealthCare Dropbox/Calvi...
...,...
188,/Users/cu135/Partners HealthCare Dropbox/Calvi...
189,/Users/cu135/Partners HealthCare Dropbox/Calvi...
190,/Users/cu135/Partners HealthCare Dropbox/Calvi...
191,/Users/cu135/Partners HealthCare Dropbox/Calvi...


# 04 - Generate Contrasts

Generate a Contrast Matrix
- This is different from the contrast matrices used in cell-means regressions such as in PALM, but it is much more powerful. 



For more information on contrast matrices, please refer to this: https://cran.r-project.org/web/packages/codingMatrices/vignettes/codingMatrices.pdf

Generally, these drastically effect the results of ANOVA. However, they are mereley a nuisance for a regression.
In essence, they assess if coefficients are significantly different

________________________________________________________________
A coding matrix (a contrast matrix if it sums to zero) is simply a way of defining what coefficients to evaluate and how to evaluate them. 
If a coefficient is set to 1 and everything else is set to zero, we are taking the mean of the coefficient's means and assessing if they significantly
deviate from zero--IE we are checking if it had a significant impact on the ability to predict the depdendent variable.
If a coefficient is set to 1, another is -1, and others are 0, we are assessing how the means of the two coefficients deviate from eachother. 
If several coefficients are 1 and several others are -1, we are assessing how the group-level means of the two coefficients deviate from eachother.
If a group of coefficients are 1, a group is -1, and a group is 0, we are only assessing how the groups +1 and -1 have differing means. 

1: This value indicates that the corresponding variable's coefficient in the model is included in the contrast. It means you are interested in estimating the effect of that variable.

0: This value indicates that the corresponding variable's coefficient in the model is not included in the contrast. It means you are not interested in estimating the effect of that variable.

-1: This value indicates that the corresponding variable's coefficient in the model is included in the contrast, but with an opposite sign. It means you are interested in estimating the negative effect of that variable.

----------------------------------------------------------------
The contrast matrix is typically a matrix with dimensions (number of contrasts) x (number of regression coefficients). Each row of the contrast matrix represents a contrast or comparison you want to test.

For example, let's say you have the following regression coefficients in your model:

Intercept, Age, connectivity, Age_interaction_connectivity
A contrast matric has dimensions of [n_predictors, n_experiments] where each experiment is a contrast

If you want to test the hypothesis that the effect of Age is significant, you can set up a contrast matrix with a row that specifies this contrast (actually an averaging vector):
```
[0,1,0,0]. This is an averaging vector because it sums to 1
```
This contrast will test the coefficient corresponding to the Age variable against zero.


If you want to test the hypothesis that the effect of Age is different from the effect of connectivity, you can set up a contrast matrix with two rows:
```
[0,1,−1,0]. This is a contrast because it sums to 0
```

Thus, if you want to see if any given effect is significant compared to the intercept (average), you can use the following contrast matrix:
```
[1,0,0,0]
[-1,1,0,0]
[-1,0,1,0]
[-1,0,0,1] actually a coding matrix of averaging vectors
```

The first row tests the coefficient for Age against zero, and the second row tests the coefficient for connectivity against zero. The difference between the two coefficients can then be assessed.
_____
You can define any number of contrasts in the contrast matrix to test different hypotheses or comparisons of interest in your regression analysis.

It's important to note that the specific contrasts you choose depend on your research questions and hypotheses. You should carefully consider the comparisons you want to make and design the contrast matrix accordingly.

- Examples:
    - [Two Sample T-Test](https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/GLM#Two-Group_Difference_.28Two-Sample_Unpaired_T-Test.29)
    - [One Sample with Covariate](https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/GLM#Single-Group_Average_with_Additional_Covariate)

In [19]:
contrast_matrix = cal_palm.generate_basic_contrast_matrix(design_matrix)

Here is a basic contrast matrix set up to evaluate the significance of each variable.
Here is an example of what your contrast matrix looks like as a dataframe: 


,Intercept,cluster_0[T.True],cluster_1[T.True],cluster_2[T.True],cluster_3[T.True],cluster_4[T.True],cluster_5[T.True],cluster_6[T.True]
0,1,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0
3,0,0,0,1,0,0,0,0
4,0,0,0,0,1,0,0,0
5,0,0,0,0,0,1,0,0
6,0,0,0,0,0,0,1,0
7,0,0,0,0,0,0,0,1


Below is the same contrast matrix, but as an array.
Copy it into a cell below and edit it for more control over your analysis.
[
    [1, 0, 0, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 1],
]


Edit Contrast Matrix Here
- The generic contrast matrix will simply check if your Betas are significantly different from the intercept (average)

In [20]:
contrast_matrix = [
    [0, 1, -1/6, -1/6, -1/6, -1/6, -1/6, -1/6],
    [0, -1/6, 1, -1/6, -1/6, -1/6, -1/6, -1/6],
    [0, -1/6, -1/6, 1, -1/6, -1/6, -1/6, -1/6],
    [0, -1/6, -1/6, -1/6, 1, -1/6, -1/6, -1/6],
    [0, -1/6, -1/6, -1/6, -1/6, 1, -1/6, -1/6],
    [0, -1/6, -1/6, -1/6, -1/6, -1/6, 1, -1/6],
    [0, -1/6, -1/6, -1/6, -1/6, -1/6, -1/6, 1],
]

Finalize Contrast Matrix

In [21]:
contrast_matrix_df = cal_palm.finalize_contrast_matrix(design_matrix=design_matrix, 
                                                    contrast_matrix=contrast_matrix) 
contrast_matrix_df

,Intercept,cluster_0[T.True],cluster_1[T.True],cluster_2[T.True],cluster_3[T.True],cluster_4[T.True],cluster_5[T.True],cluster_6[T.True]
0,0,1.000000,-0.166667,-0.166667,-0.166667,-0.166667,-0.166667,-0.166667
1,0,-0.166667,1.000000,-0.166667,-0.166667,-0.166667,-0.166667,-0.166667
2,0,-0.166667,-0.166667,1.000000,-0.166667,-0.166667,-0.166667,-0.166667
3,0,-0.166667,-0.166667,-0.166667,1.000000,-0.166667,-0.166667,-0.166667
4,0,-0.166667,-0.166667,-0.166667,-0.166667,1.000000,-0.166667,-0.166667
5,0,-0.166667,-0.166667,-0.166667,-0.166667,-0.166667,1.000000,-0.166667
6,0,-0.166667,-0.166667,-0.166667,-0.166667,-0.166667,-0.166667,1.000000


# 06 - Save the Files

Standardization during regression is critical. 
- data_transform_method='standardize' will ensure the voxelwise values are standardized
    - if you design matrix has a column called 'Dataset', the standardization will standardize values within each dataset individually, which is as should be done normally.
    - If you call data_transform_method='standardize' without having a 'Dataset' column in your design matrix, the entire collection of images will be standardized. This is potentially dangerous and misleading. Be careful, and consider not standardizing at all, or going back and adding a 'Dataset' column. 

Mask Path
- set mask_path to the path of your local brain mask which matches the resolution of the files you have collected. Typically this is an MNI 152 brain mask. 
    - download one here: https://nilearn.github.io/dev/modules/generated/nilearn.datasets.load_mni152_brain_mask.html

In [22]:
mask_path = '/Users/cu135/Software_Local/calvin_utils_project/calvin_utils_project/resources/MNI152_T1_2mm_brain_mask.nii'
data_transform_method='standardize'

In [23]:
from calvin_utils.ccm_utils.npy_utils import RegressionNPYPreparer
preparer = RegressionNPYPreparer(
    design_matrix=design_matrix,
    contrast_matrix=contrast_matrix_df,
    outcome_matrix=outcome_matrix,
    out_dir=out_dir,
    mask_path=mask_path,
    exchangeability_blocks=None,   # or your DataFrame
    data_transform_method=data_transform_method
)
dataset_dict, json_path = preparer.run()

Design matrix saved to: /Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/raynor_network_mapping/results/corbetta_cluster/umapsV2/cluster_unique/main/design_matrix.npy
Contrast matrix saved to: /Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/raynor_network_mapping/results/corbetta_cluster/umapsV2/cluster_unique/main/contrast_matrix.npy


Loading NIFTI files: 100%|██████████| 193/193 [00:02<00:00, 84.00it/s]


Stacked NIFTI data saved to: /Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/raynor_network_mapping/results/corbetta_cluster/umapsV2/cluster_unique/main/niftis.npy
Dataset dictionary saved to: /Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/raynor_network_mapping/results/corbetta_cluster/umapsV2/cluster_unique/dataset_dict.json


# 07 - Run the Regression

Mask Path
- set mask_path to the path of your local brain mask which matches the resolution of the files you have collected. Typically this is an MNI 152 brain mask. 
    - download one here: https://nilearn.github.io/dev/modules/generated/nilearn.datasets.load_mni152_brain_mask.html

In [24]:
from calvin_utils.ccm_utils.npy_regression import RegressionNPYAnalysis
reg = RegressionNPYAnalysis(data_dict_path=json_path,
                      n_permutations=1000, 
                      out_dir=out_dir,
                      fwe=True,
                      max_stat_method="pseudo_var_smooth",
                      mask_path=mask_path,
                      verbose=False)
results = reg.run()
print("Voxelwise FWE p-values shape:", results["voxelwise_p_values"].shape)

There are 193 subjects
There are 8 covariates
There are 228483 voxels
X (observations, covariates): (193, 8)
Y (voxels, observations): (228483, 193)

 Contrast matrix: 
 f[[ 0.          1.         -0.16666667 -0.16666667 -0.16666667 -0.16666667
  -0.16666667 -0.16666667]
 [ 0.         -0.16666667  1.         -0.16666667 -0.16666667 -0.16666667
  -0.16666667 -0.16666667]
 [ 0.         -0.16666667 -0.16666667  1.         -0.16666667 -0.16666667
  -0.16666667 -0.16666667]
 [ 0.         -0.16666667 -0.16666667 -0.16666667  1.         -0.16666667
  -0.16666667 -0.16666667]
 [ 0.         -0.16666667 -0.16666667 -0.16666667 -0.16666667  1.
  -0.16666667 -0.16666667]
 [ 0.         -0.16666667 -0.16666667 -0.16666667 -0.16666667 -0.16666667
   1.         -0.16666667]
 [ 0.         -0.16666667 -0.16666667 -0.16666667 -0.16666667 -0.16666667
  -0.16666667  1.        ]] 

Using Family-wise error correction: True with method: pseudo_var_smooth

 Saving results to /Users/cu135/Partners HealthCare Dr

/Users/cu135/Software_Local/calvin_utils_project/calvin_utils_project/calvin_utils/ccm_utils/npy_regression.py:124: RuntimeWarning: invalid value encountered in divide
  t_values = beta / se_beta                                       # (n_cov, n_voxels) <- (n_cov, n_voxels) / (n_cov, n_voxels)
/Users/cu135/Software_Local/calvin_utils_project/calvin_utils_project/calvin_utils/ccm_utils/npy_regression.py:179: RuntimeWarning: invalid value encountered in divide
  contrast_t = contrast_est / contrast_se                 # (n_voxels,)
Permutations: 100%|██████████| 1000/1000 [05:15<00:00,  3.17it/s]


Voxelwise FWE p-values shape: (7, 228483)


View and Save the Maps

In [25]:
# Save and visualize results
reg.save_and_visualize_results(verbose=True)  # Change to False to disable visualization

Saving results...


/Users/cu135/Software_Local/calvin_utils_project/.venv_calvin/lib/python3.10/site-packages/numpy/core/fromnumeric.py:771: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)
/Users/cu135/Software_Local/calvin_utils_project/.venv_calvin/lib/python3.10/site-packages/numpy/core/fromnumeric.py:771: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)
/Users/cu135/Software_Local/calvin_utils_project/.venv_calvin/lib/python3.10/site-packages/numpy/core/fromnumeric.py:771: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)
/Users/cu135/Software_Local/calvin_utils_project/.venv_calvin/lib/python3.10/site-packages/numpy/core/fromnumeric.py:771: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, o

All results saved.
